In [1]:
import sys
sys.path.insert(0, '../')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from unet.model import GeneralUNet

In [2]:
# Check if CUDA (GPU support) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [3]:
in_channels = 1
size = 2
conv3d = True
complex = 2

# Create dummy Volumetric dataset
x_dummy = torch.rand(10, in_channels, 240, 240, 155).to(device)  # Move tensor to the device (GPU if available)
y_dummy = torch.randint(0, 2, (10, in_channels, 240, 240, 155)).float().to(device)  # Move tensor to the device

dataset = TensorDataset(x_dummy, y_dummy)
dataloader = DataLoader(dataset, batch_size=2)

# Initialize the model and move it to the GPU
model = GeneralUNet(in_channels, 3,2,2,0.1,1,1, conv3d, size, complex).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check if GPU is available
if torch.cuda.is_available():
    # Get the available memory
    available_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)  # Total memory in GB
    print(f"Available GPU Memory: {available_memory:.2f} GB")
else:
    print("GPU is not available.")

Encoder: adding convDouble
Adding Double Conv layer, in: 1, out: 4
adding conv3d layer with in_channels: 1, out_channels: 4
adding conv3d layer with in_channels: 4, out_channels: 4

Encoder: adding convDouble
Adding Double Conv layer, in: 4, out: 8
adding conv3d layer with in_channels: 4, out_channels: 8
adding conv3d layer with in_channels: 8, out_channels: 8

Bottle neck appended, in: 8, out: 16

Decoder: adding convDouble
Adding Double Conv layer, in: 16, out: 8
adding conv3d layer with in_channels: 16, out_channels: 8
adding conv3d layer with in_channels: 8, out_channels: 8

Decoder: adding convDouble
Adding Double Conv layer, in: 8, out: 4
adding conv3d layer with in_channels: 8, out_channels: 4
adding conv3d layer with in_channels: 4, out_channels: 4

added last 1x1x1 conv layer
Available GPU Memory: 8.00 GB


In [4]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

21485
21485


In [5]:
# Training loop for one epoch
model.train()  # Set the model to training mode
for batch_idx, (data, target) in enumerate(dataloader):
    data, target = data.to(device), target.to(device)  # Move data and target to device
    optimizer.zero_grad()  # Zero the gradients
    output = model(data)  # Forward pass: Predict
    loss = criterion(output, target)  # Compute loss
    loss.backward()  # Backward pass: Compute gradient
    optimizer.step()  # Update weights

    print(f'Batch {batch_idx}, Loss {loss.item()}')  # Print loss for each batch

input features shape: torch.Size([2, 1, 240, 240, 155])
Encoder Block Forward
single conv, input shape is: torch.Size([2, 1, 240, 240, 155])
output shape is: torch.Size([2, 4, 240, 240, 155])
single conv, input shape is: torch.Size([2, 4, 240, 240, 155])
output shape is: torch.Size([2, 4, 240, 240, 155])
post_conv_features.shape: torch.Size([2, 4, 240, 240, 155])
shape is: torch.Size([2, 4, 120, 120, 77])
Encoder Block Forward
single conv, input shape is: torch.Size([2, 4, 120, 120, 77])
output shape is: torch.Size([2, 8, 120, 120, 77])
single conv, input shape is: torch.Size([2, 8, 120, 120, 77])
output shape is: torch.Size([2, 8, 120, 120, 77])
post_conv_features.shape: torch.Size([2, 8, 120, 120, 77])
shape is: torch.Size([2, 8, 60, 60, 38])

Completed Encoder Block Forward Prop
x.shape: torch.Size([2, 8, 60, 60, 38])
skip_connection.shape is: torch.Size([2, 4, 240, 240, 155])
skip_connection.shape is: torch.Size([2, 8, 120, 120, 77])
Bottleneck Forward
Decoder Block Forward
single 